# Nested Dichotomous Data

## Quickstart

To run a nested dichotomous dataset:

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# tag: remove-cell applied

In [ ]:
import pybmds

dataset = pybmds.NestedDichotomousDataset(
    name="Nested Dataset",
    dose_units="ppm",
    doses= [
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        100, 100, 100, 100, 100, 100, 100, 100, 100
    ],
    litter_ns = [
        16, 9, 15, 14, 13, 9, 10, 14, 10, 11,
        14, 9, 14, 9, 13, 12, 10, 10, 11, 14,
        11, 11, 14, 11, 10, 11, 10, 15, 7, 14,
        11, 14, 12, 13, 12, 14, 11, 8, 10
    ],
    incidences = [
        1, 1, 2, 3, 3, 0, 2, 2, 1, 2,
        4, 5, 6, 2, 6, 3, 1, 2, 4, 3,
        4, 5, 5, 4, 5, 4, 5, 6, 2, 4,
        6, 6, 8, 7, 8, 6, 6, 5, 4
    ],
    litter_covariates = [
        16, 9, 15, 14, 13, 9, 10, 14, 10, 11,
        14, 9, 14, 9, 13, 12, 10, 10, 11, 14,
        11, 11, 14, 11, 10, 11, 10, 15, 7, 14,
        11, 14, 12, 13, 12, 14, 11, 8, 10
    ]
)

# create a BMD session
session = pybmds.Session(dataset=dataset)

# add all default models
session.add_default_models()

# execute the session
session.execute()

# recommend a best-fitting model
session.recommend()

if session.recommended_model is not None:
    display(session.recommended_model.plot())
    print(session.recommended_model.text())

# save excel report
df = session.to_df()
df.to_excel("output/nd-report.xlsx")

# save to a word report
report = session.to_docx()
report.save("output/nd-report.docx")

## Nested dichotomous dataset

Creating a nested dichotomous dataset requires a list of doses, litter ns, incidence, and litter covariates. All lists must have must have the same number of items, with the total items equal to the total number of litters.

You can also add optional attributes, such as `name`, `dose_name`, `dose_units`, `response_name`, `response_units`, etc.

In [ ]:
dataset = pybmds.NestedDichotomousDataset(
    name="ChemX",
    dose_name="Oral Gavage",
    dose_units="mg/kg/d",
    doses= [
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        100, 100, 100, 100, 100, 100, 100, 100, 100
    ],
    litter_ns = [
        16, 9, 15, 14, 13, 9, 10, 14, 10, 11,
        14, 9, 14, 9, 13, 12, 10, 10, 11, 14,
        11, 11, 14, 11, 10, 11, 10, 15, 7, 14,
        11, 14, 12, 13, 12, 14, 11, 8, 10
    ],
    incidences = [
        1, 1, 2, 3, 3, 0, 2, 2, 1, 2,
        4, 5, 6, 2, 6, 3, 1, 2, 4, 3,
        4, 5, 5, 4, 5, 4, 5, 6, 2, 4,
        6, 6, 8, 7, 8, 6, 6, 5, 4
    ],
    litter_covariates = [
        16, 9, 15, 14, 13, 9, 10, 14, 10, 11,
        14, 9, 14, 9, 13, 12, 10, 10, 11, 14,
        11, 11, 14, 11, 10, 11, 10, 15, 7, 14,
        11, 14, 12, 13, 12, 14, 11, 8, 10
    ]
)

dataset.plot()

## Single model fit

If you want to fit only one model to your dataset, you can fit the specific model to the dataset and print the results such as the BMD, BMDL, BMDU, p-value, AIC, etc.

For example, to execute the Nested Logistic model:

In [ ]:
from pybmds.models import nested_dichotomous

model = nested_dichotomous.NestedLogistic(dataset)
model.execute()
model.plot()

An output report can be generated after execution:

In [ ]:
print(model.text())

### Change input settings


The default settings for a use a BMR of 10% Extra Risk and a 95% confidence interval. Settings can be edited as shown below when executing a single model:

In [ ]:
model = nested_dichotomous.NestedLogistic(dataset, settings={
    "bmr": 0.15,
    "bmr_type": pybmds.DichotomousRiskType.AddedRisk
})
print(model.settings.tbl())

BMR settings are similar to standard dichotomous models. Nested Dichotomous models can be run with different mdoeling settings for  the Litter Specific Covariance (lsc) and the Intralitter Correlation (ilc):

In [ ]:
from pybmds.types.nested_dichotomous import LitterSpecificCovariate, IntralitterCorrelation

model = nested_dichotomous.NestedLogistic(dataset, settings={
    "litter_specific_covariate": LitterSpecificCovariate.Unused,
    "intralitter_correlation": IntralitterCorrelation.Zero,
})
print(model.settings.tbl())

Choices for `LitterSpecificCovariate` include:

In [ ]:
for item in LitterSpecificCovariate:
    print(f"{item.name}: {item.value}")

Choices for `IntralitterCorrelation` include:

In [ ]:
for item in IntralitterCorrelation:
    print(f"{item.name}: {item.value}")

### Change parameter settings

To preview initial parameter settings:

In [ ]:
model = nested_dichotomous.NestedLogistic(dataset)
print(model.priors_tbl())

Initial parameter settings can also can be modified:

In [ ]:
model.settings.priors.update('a', initial_value=2, min_value=-10, max_value=10)
model.settings.priors.update('phi1', initial_value=10, min_value=5, max_value=100)
print(model.priors_tbl())

## Multiple model fit (sessions) and model recommendation

A Session in pybmds allows for multiple different models to be executed and potentially compared for model recommendation and selection.

A common pattern may be to add multiple versions of the same model with varying settings for the litter specific covariate and intralitter correlation.  In the example below, we run four permutation of the Nested Logistic model:

In [ ]:
session = pybmds.Session(dataset=dataset)

for lsc in [LitterSpecificCovariate.Unused, LitterSpecificCovariate.OverallMean]:
    for ilc in [IntralitterCorrelation.Zero, IntralitterCorrelation.Estimate]:
        session.add_model(
            pybmds.Models.NestedLogistic,
            settings={
                "bmr": 0.15,
                "litter_specific_covariate": lsc,
                "intralitter_correlation": ilc,
            }
        )

session.execute()
session.plot()

Model recommendation can be enabled, and if a recommendation can be mode, you can view outputs:

In [ ]:
session.recommend()

if session.recommended_model is not None:
    display(session.recommended_model.plot())
    print(session.recommended_model.text())

### Select a best fitting model

The `pybmds` package may recommend a best fitting model based on a decision tree, but expert judgment may be required for model selection.

You can select any model; in this example we can agree agree with the recommended model:

In [ ]:
session.select(model=session.recommended_model, notes="Lowest AIC; recommended model")

Generated outputs (Excel, Word, JSON) would include model selection information.